In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
from langchain_community.document_loaders import Docx2txtLoader, PDFPlumberLoader
from pathlib import Path
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import hashlib, time

In [2]:
import pathlib, subprocess

def docx_to_pdf_linux(input_docx: str, output_pdf: str | None = None) -> str:
    input_docx = os.path.abspath(input_docx)
    out_dir = os.path.abspath(os.path.dirname(output_pdf)) if output_pdf else os.path.dirname(input_docx)
    os.makedirs(out_dir, exist_ok=True)

    # LibreOffice положит PDF в out_dir с тем же именем
    subprocess.run(
        ["soffice", "--headless", "--convert-to", "pdf", "--outdir", out_dir, input_docx],
        check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    if output_pdf is None:
        output_pdf = os.path.join(out_dir, pathlib.Path(input_docx).with_suffix(".pdf").name)
    return output_pdf

In [3]:
client = QdrantClient(
    host="localhost",
    port=6333
)

In [4]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='norma_ai_collection_ru-en-RoSBERTa')])

In [28]:
model_name = "ai-forever/ru-en-RoSBERTa"
model_kwargs = {"device":"cpu"}
encode_kwargs = {'normalize_embeddings': True, 'batch_size': 128}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

def payload_for(doc):
    text = doc.page_content
    source_path =doc.metadata['source']
    return {
        "source": source_path,
        "filename": os.path.basename(source_path),
        "doc_type": os.path.splitext(source_path)[1].lower().strip("."),
        "page": doc.metadata.get("page"),
        "title": doc.metadata.get("Title"),
        "created_at": int(time.time()),
        "sha1": hashlib.sha1(text.encode("utf-8", "ignore")).hexdigest(),
        # "lang": detect_lang(text)  # см. п.4
    }

In [ ]:
# emb_dim = len(embeddings_model.embed_query("probe"))

In [ ]:
# client.create_collection(
#     collection_name="norma_ai_collection_ru-en-RoSBERTa",
#     vectors_config=VectorParams(size=emb_dim, distance=Distance.COSINE),
# )

True

In [7]:
all_documents = os.listdir('../data')
paths = '/srv/norma_ai/norma_ai_project/data/'

loaded_docs = []
broken_files = []


for i, p in enumerate(tqdm(all_documents, desc="Enumerating")):
    print(f'Файл № {i+1}')
    current_path = paths + p
    print(f"Trying to load file: {p}")
    if current_path.lower().endswith(".docx"):
        try:
            out_path = docx_to_pdf_linux(current_path, current_path[:current_path.rfind('.')] + '.pdf')
            loaded_docs.extend(PDFPlumberLoader(out_path).load())
            print(f"The download was successful: {p}")
            print('-'*50)
        except Exception as e:
            print(f'Нерабочий файл: {p}')
            broken_files.append(p)
            print('-'*50)
            continue
    elif current_path.lower().endswith(".pdf"):
        path = Path(current_path)
        try:
            loaded_docs.extend(PDFPlumberLoader(str(path)).load())
            print(f"The download was successful: {p}")
            print('-'*50)
        except Exception as e:
            print(f'Нерабочий файл: {p}')
            broken_files.append(p)
            print('-'*50)
            continue
    else:
        broken_files.append(p)

Enumerating:   0%|          | 0/45 [00:00<?, ?it/s]

Файл № 1
Trying to load file: 5. ФЗ 69 О пожарной безопасности.pdf


Enumerating:   2%|▏         | 1/45 [00:04<02:57,  4.03s/it]

The download was successful: 5. ФЗ 69 О пожарной безопасности.pdf
--------------------------------------------------
Файл № 2
Trying to load file: СП 68.13330.2017 Приемка в эксплуатацию законченных строительство объектов.pdf


Enumerating:   4%|▍         | 2/45 [00:07<02:42,  3.78s/it]

...

The download was successful: 7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf
--------------------------------------------------
Файл № 44
Trying to load file: 7. ФЗ 116 О промышленной безопасности ОПО.pdf


Enumerating:  98%|█████████▊| 44/45 [06:32<00:08,  8.47s/it]

The download was successful: 7. ФЗ 116 О промышленной безопасности ОПО.pdf
--------------------------------------------------
Файл № 45
Trying to load file: 15. Приказ РТН № 519 ФНИП Требования к производству сварочных работ на опасных производственных объектах.pdf


Enumerating: 100%|██████████| 45/45 [06:34<00:00,  8.76s/it]

The download was successful: 15. Приказ РТН № 519 ФНИП Требования к производству сварочных работ на опасных производственных объектах.pdf
--------------------------------------------------


In [8]:
len(loaded_docs)

3231

In [ ]:
loaded_docs[100].

Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/19. Приказ РТН № 536 ФНИП Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением.pdf', 'file_path': '/srv/norma_ai/norma_ai_project/data/19. Приказ РТН № 536 ФНИП Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением.pdf', 'page': 100, 'total_pages': 198, 'Title': 'Приказ Ростехнадзора от 15.12.2020 N 536"Об утверждении федеральных норм и правил в области промышленной безопасности "Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением"(Зарегистрировано в Минюсте Росси', 'Author': 'Evgeniy Ostromecky', 'Creator': 'Writer', 'Producer': 'LibreOffice 24.2', 'CreationDate': "D:20250827112706Z'"}, page_content='Приказ Ростехнадзора от 15.12.2020 N 536\nДокумент предоставлен КонсультантПлюс\n"Об утверждении федеральных норм и правил в области\nДата сохранения: 14.02.2022\nпромышленной

In [43]:
payload_for(loaded_docs[19])

{'source': '/srv/norma_ai/norma_ai_project/data/ГОСТ Р 55990-2014.pdf',
 'filename': 'ГОСТ Р 55990-2014.pdf',
 'doc_type': 'pdf',
 'page': 10,
 'title': None,
 'created_at': 1756290001,
 'sha1': '57ee93e62f008358b6de22dc64d47d75eabaf33c'}

In [ ]:
# # from langchain_text_splitters import CharacterTextSplitter

# # splitter_1 = CharacterTextSplitter.from_tiktoken_encoder(
# #     encoding_name="cl100k_base", chunk_size=480, chunk_overlap=80
# # )
# docs_1 = splitter_1.split_documents(loaded_docs)

In [24]:
# Чанкинг под регламенты (чуть крупнее блоки)
# splitter_1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs_1 = splitter_1.split_documents(loaded_docs)



splitter_2 = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
docs_2 = splitter_2.split_documents(loaded_docs)

In [25]:
# (опционально) добавить базовые метаданные
for d in docs_1:
    d.metadata.setdefault("source", d.metadata.get("source") or "local")

In [26]:
for d in docs_2:
    d.metadata.setdefault("source", d.metadata.get("source") or "local")

In [29]:
vs = QdrantVectorStore(
    client=client, 
    collection_name="norma_ai_collection_ru-en-RoSBERTa",
    embedding=embeddings_model,
)

In [30]:
from tqdm.auto import tqdm

BATCH = 64
total = len(docs_2)

with tqdm(total=total, desc="Adding to Qdrant", unit="doc") as pbar:
    for i in range(0, total, BATCH):
        batch = docs_2[i:i+BATCH]
        vs.add_documents(batch)          # эмбеддинг + upsert в Qdrant
        pbar.update(len(batch))

Adding to Qdrant:   0%|          | 0/8382 [00:00<?, ?doc/s]

In [36]:
query = "На какой максимальной высоте разрешается устанавливать штурвал ЗРА без применения площадок обслуживания"
results_with_scores = vs.similarity_search_with_score(
    query,
    k=7
)

for i, (doc, score) in enumerate(results_with_scores):
    print(f"\n--- Результат {i+1} ---")
    print(f"Описание: {doc.page_content[:]}")


--- Результат 1 ---
Описание: быть не менее 1 м. Люлька должна быть застрахована от падения.
350. При механизированном осуществлении спускоподъемных операций без участия верхового
КонсультантПлюс
www.consultant.ru Страница 37 из 312
надежная правовая поддержка

--- Результат 2 ---
Описание: "Руководство по безопасности "Рекомендации по
Документ предоставленКонсультантПлюс
техническому диагностированию сварных вертикальных
Дата сохранения: 09.01.2024
цилиндрических резервуа...
осадки основания) рекомендуется ежегодно проводить нивелирование окрайки днища в абсолютных
отметках и результаты заносить в журнал нивелирования окрайки днища.
26. Нивелирование стационарных стальных и алюминиевых крыш проводится для определения
отклонений высотных отметок несущих конструкций стальных и алюминиевых стационарных крыш от
значений, приведенных в проектной документации, в случае выявления видимых деформаций по
результатам ВИК.
27. Нивелирование стальных крыш проводится с площадок на крыше резервуара

In [32]:
# Проверочный блок для PDF

current_path = '/srv/norma_ai/norma_ai_project/data/7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf'
path = Path(current_path)
loaded_docs_test = PDFPlumberLoader(str(path)).load()

In [18]:
loaded_docs_test

[Document(metadata={'source': '/srv/norma_ai/norma_ai_project/temporary_data/1. ТР ЕАЭС 049-2020 О требованиях к магистральным трубопроводам для транспортирования жидких и газообразных углеводородов.pdf', 'file_path': '/srv/norma_ai/norma_ai_project/temporary_data/1. ТР ЕАЭС 049-2020 О требованиях к магистральным трубопроводам для транспортирования жидких и газообразных углеводородов.pdf', 'page': 0, 'total_pages': 33, 'Title': 'Решение Совета Евразийской экономической комиссии от 23.12.2020 N 121"О техническом регламенте Евразийского экономического союза "О требованиях к магистральным трубопроводам для транспортирования жидких и газообразных углеводородов"(вместе с "ТР ЕАЭС 49/2', 'Author': 'Evgeniy Ostromecky', 'Creator': 'Writer', 'Producer': 'LibreOffice 24.2', 'CreationDate': "D:20250827104938Z'"}, page_content='Решение Совета Евразийской\nэкономической комиссии от 23.12.2020 N\n121\n"О техническом регламенте Евразийского\nэкономического союза "О требованиях к\nмагистральным трубо

In [33]:
loaded_docs_test

[Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf', 'file_path': '/srv/norma_ai/norma_ai_project/data/7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf', 'page': 0, 'total_pages': 222, 'CreationDate': 'D:20240109085328Z', 'Creator': 'Версия 4023.00.09© 1992 - 2023 КонсультантПлюс Сборка 672292', 'ModDate': "D:20250826142739+03'00'", 'Producer': 'КонсультантПлюс', 'Title': '"Документация по промышленной безопасности"\n(Тихомирова Л.А.)\n(Подготовлен для системы КонсультантПлюс, 2020)'}, page_content='"Документация по промышленной\nбезопасности"\n(Тихомирова Л.А.)\n(Подготовлен для системы КонсультантПлюс,\n2020)\nДокумент предоставлен КонсультантПлюс\nwww.consultant.ru\nДата сохранения: 09.01.2024\n'),
 Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf', 'file_path': '/srv/norma_ai/norma_ai_project/data/7. ФЗ 116 О промышленной безопас

In [35]:
loaded_docs_test[31].page_content

'"Документация по промышленной безопасности"\nДокумент предоставленКонсультантПлюс\n(Тихомирова Л.А.)\nДата сохранения: 09.01.2024\n(Подготовлен для системы КонсультантПлюс, 2020)\nКак было отмечено ранее, деятельность в области промышленной безопасности охватывает далеко\nне каждую хозяйственную деятельность, не каждый промышленный объект и оборудование и не каждый\nхозяйствующий субъект. Объединяет обозначенные виды деятельности то, что все они связаны с\nопасными производственными объектами. Поэтому в определении необходимости соблюдения\nтребований промышленной безопасности важно определить статус промышленного объекта как опасного\nпроизводственного объекта.\nПравовой статус опасных производственных объектов определяется в статье 2 Федерального\nзакона от 21.07.1997 N 116-ФЗ "О промышленной безопасности опасных производственных объектов".\nПризнаки опасности, определяющие необходимость отнесения объекта к категории опасного\nпроизводственного объекта, устанавливаются приложением 1